In [1]:
import sqlite3
import pandas as pd
import re
import keras
from unicodedata import normalize
import utils.functions as custom_functions

Using TensorFlow backend.


In [2]:
import imp
imp.reload(custom_functions)

<module 'utils.functions' from '/home/migueljaime/generative_language_model/utils/functions.py'>

In [3]:
sqlite_file = 'data/database.sqlite'
db = custom_functions.create_connection(sqlite_file)

In [6]:
# get a small sample
small_sample = custom_functions.get_sample(nrows=50000, db=db)
display(small_sample)

# save as csv
small_sample.to_csv('data/reddit_small_sample.csv', index = False)

querying for sample
query for sample finished
getting parents
obtaining parents
done with parents
checking for dupes
done with dupes
getting parents completed
basic preprocessing
converting parent ids
we done


,subreddit,ups,downs,score,body,id,name,link_id,parent_id
0,AskReddit,0,0,0,theres a pretty big difference between 35 ciga...,cqugd30,t1_cqugd30,t3_34ffo5,cqu6659
1,worldnews,6,0,6,i would expect to be shot in the head if i thr...,cqugdws,t1_cqugdws,t3_34fiat,cqub2yi
2,dataisbeautiful,4,0,4,well india nigeria brazil and indonesia arent ...,cqugeb8,t1_cqugeb8,t3_34dd4h,cqtq6nc
3,Texans,2,0,2,lets go texans draft us a badass,cqugegn,t1_cqugegn,t3_34dxky,34dxky
4,nfl,1,0,1,here come the boos,cqugesn,t1_cqugesn,t3_34gjfz,34gjfz
5,WTF,38,0,38,this is for fucking kidsedit this is for kids,cqugfcl,t1_cqugfcl,t3_34g65j,cqug559
6,serialkillers,1,0,1,i really enjoyed fargo and imaginary internet ...,cqugfdt,t1_cqugfdt,t3_34fewy,cqufyyy
7,nfl,1,0,1,shaky cam fucking with all the drunk fans at home,cqugfje,t1_cqugfje,t3_34gjek,34gjek
8,TPPKappa,5,0,5,it was fine just kinda hanging out by myself h...,cqugfyw,t1_cqugfyw,t3_34gmcz,34gmcz
9,KCRoyals,2,0,2,that is some hot ass add youve got there,cqugg2o,t1_cqugg2o,t3_34gi5o,cqugfg0


In [5]:
# TODO
# get bigger samples, use remove duplicates, and merge

# neet to create a list of ids from sample_df_1
#df_2, ex_id = remove_duplicates(sample_df_2, set(sample_df_1.id.unique()))